In [1]:
#from sklearn.preprocessing import MinMaxScaler
#import sqlalchemy
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
#import statsmodels.api as sm
#import yfinance as yf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Se carga la base de datos que contiene cada tweet con la calificación obtenida por los dos métodos propuestos para el análisis de sentimientos.

In [10]:
# Carga de calificaciones obtenidas por los dos métodos propuestos para cada Tweet de Apple
sent_appl = pd.read_csv('/content/drive/Shareddrives/Data Minning - Natural Language Procesing/Datos_Sentimiento/AAPL_completo.csv', sep=';', header=0)

In [11]:
sent_appl.head()

,tweet_id,Original,Procesado,Label_1,Score_1,Label_4,Score_4
0,550803612197457920,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,toptickertweets sentiqua...,NEGATIVE,0.996150,5 stars,0.812136
1,550803610825928706,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,sentishiftup sentiquant ...,NEGATIVE,0.991630,5 stars,0.237722
2,550803225113157632,Rumor Roundup: What to expect when you're expe...,rumor roundup what to expect when you re expe...,NEGATIVE,0.824964,3 stars,0.287090
3,550802957370159104,"An $AAPL store line in Sapporo Japan for the ""...",an store line in sapporo japan for the luck...,NEGATIVE,0.995857,5 stars,0.413658
4,550802855129382912,$AAPL - Will $AAPL Give Second entry opportuni...,will give second entry opportunity,POSITIVE,0.998913,4 stars,0.414050


Recodificación de las calificaciones de sentimientos para cada Tweet.

In [12]:
# Recodificación de las categorias del análisis
sent_appl.Label_1 = sent_appl.Label_1.replace(['NEGATIVE','POSITIVE'],[-1,1])
sent_appl.Label_4 = sent_appl.Label_4.replace(['1 star','2 stars','3 stars','4 stars','5 stars'],[-1,-0.5,0,0.5,1])

# Añade columnas
# Se establecen 0.95 y 0.5 como scores que deben superar los métodos 1 y 4 para considerarse validos 
sent_appl['Score_1_ind']=(sent_appl['Score_1']>0.95)*1
sent_appl['Score_4_ind']=(sent_appl['Score_4']>0.5)*1

#Calificacion combinada
# l1:label_1; l4:label_4; s1:score_1; s4:score_4; I1:1 si s1>\alpha; I4:1 si s4>\alpha (0 en otro caso).
# sum_label_rep = I1*l1+I4*l4
sent_appl['Calificacion']=(sent_appl['Score_1_ind']*sent_appl['Label_1'])+(sent_appl['Score_4_ind']*sent_appl['Label_4'])

Debido al procesamiento algunos tweets quedan vacios, por lo tanto se identifican.

In [13]:
# Identificar espacios en blanco de 'Procesado' y poner 'vacio' en cada espacio
# Hay hasta 21   
for i in range(0, 220):
  sent_appl.Procesado = sent_appl.Procesado.replace([str(' '*i)],['vacio'])#+str(i)])
sum(sent_appl.Procesado=='vacio') 

4112

Se construye Califiación ajustada a partir de la Califiación planteada anteriormente.

In [14]:
# DataFrame con la caificación e identificadores de positividad o negatividad

#sent_appl_no_vacio = sent_appl[sent_appl.Original!='vacio']
cal_APPL = sent_appl
#cal_APPL = sent_appl.rename(columns={'sum_label_rep':'Calificación'})
cal_APPL = cal_APPL[['tweet_id', 'Procesado', 'Calificacion','Label_1', 'Score_1', 'Label_4', 'Score_4']]
cal_APPL['Calificacion_ajus']=cal_APPL['Calificacion'].replace([2,1.5,1,0.5,0,-0.5,-1,-1.5,-2],[1,1,1,0,0,0,-1,-1,-1])
cal_APPL['Ind_positivo']=cal_APPL['Calificacion_ajus'].replace([1,0,-1],[1,0,0])
cal_APPL['Ind_negativo']=cal_APPL['Calificacion_ajus'].replace([1,0,-1],[0,0,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
cal_APPL.head()

,tweet_id,Procesado,Calificacion,Label_1,Score_1,Label_4,Score_4,Calificacion_ajus,Ind_positivo,Ind_negativo
0,550803612197457920,toptickertweets sentiqua...,0.0,-1,0.996150,1.0,0.812136,0.0,0.0,0.0
1,550803610825928706,sentishiftup sentiquant ...,-1.0,-1,0.991630,1.0,0.237722,-1.0,0.0,1.0
2,550803225113157632,rumor roundup what to expect when you re expe...,0.0,-1,0.824964,0.0,0.287090,0.0,0.0,0.0
3,550802957370159104,an store line in sapporo japan for the luck...,-1.0,-1,0.995857,1.0,0.413658,-1.0,0.0,1.0
4,550802855129382912,will give second entry opportunity,1.0,1,0.998913,0.5,0.414050,1.0,1.0,0.0


In [ ]:
#cal_APPL.head()
import os 
import pandas as pd

path = "/content/drive/Shareddrives/Data Minning - Natural Language Procesing/Datos_Sentimiento/" # Path to the folder you wish to keep your file under
file_name = os.path.join(path,'Calif_AAPL.csv')
cal_APPL.to_csv(file_name, sep=';',index=False)

Se carga la base con los tweets y las calificaciones y se cruza con los tweets de la base de datos en MariaDB.

In [9]:
Calif_AAPL = pd.read_csv('/content/drive/Shareddrives/Data Minning - Natural Language Procesing/Datos_Sentimiento/Calif_AAPL.csv',sep=';',header=0)

In [ ]:
Company_Tweet = pd.read_csv('/content/drive/Shareddrives/Data Minning - Natural Language Procesing/Datos_Sentimiento/Company_Tweet_MariaDB.csv',sep=',',header=0)
Tweet = pd.read_csv('/content/drive/Shareddrives/Data Minning - Natural Language Procesing/Datos_Sentimiento/Tweet_MariaDB.csv',sep=',',header=0)

In [ ]:
# Unión tabla company_tweet con la de tweet para corroborar que concuerden 
Company_Tweet_Tweet = Company_Tweet.merge(Tweet,how= 'inner',on='tweet_id')
Company_Tweet_Tweet = Company_Tweet_Tweet[Company_Tweet_Tweet.ticker_symbol=='AAPL']

In [ ]:
Tweet_all = Calif_AAPL.merge(Company_Tweet_Tweet, how='inner', on='tweet_id')

In [ ]:
Tweet_all.head()

,tweet_id,Procesado,Calificacion,Label_1,Score_1,Label_4,Score_4,Calificacion_ajus,Ind_positivo,Ind_negativo,ticker_symbol,writer,post_date,body,retweet_num,like_num
0,550803612197457920,toptickertweets sentiqua...,0.0,-1,0.996150,1.0,0.812136,0.0,0.0,0.0,AAPL,SentiQuant,2015-01-01,#TOPTICKERTWEETS $AAPL $IMRS $BABA $EBAY $AMZN...,0,1
1,550803610825928706,sentishiftup sentiquant ...,-1.0,-1,0.991630,1.0,0.237722,-1.0,0.0,1.0,AAPL,SentiQuant,2015-01-01,#SENTISHIFTUP $K $FB $GOOGL $GS $GOLD $T $AAPL...,0,1
2,550803225113157632,rumor roundup what to expect when you re expe...,0.0,-1,0.824964,0.0,0.287090,0.0,0.0,0.0,AAPL,MacHashNews,2015-01-01,Rumor Roundup: What to expect when you're expe...,0,0
3,550802957370159104,an store line in sapporo japan for the luck...,-1.0,-1,0.995857,1.0,0.413658,-1.0,0.0,1.0,AAPL,WaltLightShed,2015-01-01,"An $AAPL store line in Sapporo Japan for the ""...",4,4
4,550802855129382912,will give second entry opportunity,1.0,1,0.998913,0.5,0.414050,1.0,1.0,0.0,AAPL,2waystrading,2015-01-01,$AAPL - Will $AAPL Give Second entry opportuni...,0,0


In [ ]:
#cal_APPL.head()
#import os 
#import pandas as pd

path = "/content/drive/Shareddrives/Data Minning - Natural Language Procesing/Datos_Sentimiento/" # Path to the folder you wish to keep your file under
file_name = os.path.join(path,'Tweet_all_AAPL.csv')
Tweet_all.to_csv(file_name, sep=';',index=False)

Tweet_all_AAPL es la base de datos que contiene cada tweet con la totalidad de información disponible.

In [ ]:
Tweet_all_AAPL = pd.read_csv('/content/drive/Shareddrives/Data Minning - Natural Language Procesing/Datos_Sentimiento/Tweet_all_AAPL.csv',sep=';',header=0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
print(Tweet_all_AAPL.shape)
print(Calif_AAPL.shape)
print(Company_Tweet_Tweet.shape)

(1424002, 11)
(1424013, 5)
(1425001, 7)


In [ ]:
print(sum(Company_Tweet_Tweet.retweet_num==0))
print(sum(Company_Tweet_Tweet.retweet_num==1))
print(sum(Company_Tweet_Tweet.retweet_num==2))
print(sum(Company_Tweet_Tweet.retweet_num==3))
print(sum(Company_Tweet_Tweet.retweet_num==4))
print(sum(Company_Tweet_Tweet.retweet_num==5))
print(sum(Company_Tweet_Tweet.retweet_num==6))
print(sum(Company_Tweet_Tweet.retweet_num==7))
print(sum(Company_Tweet_Tweet.retweet_num==8))
print(sum(Company_Tweet_Tweet.retweet_num==9))
print(sum(Company_Tweet_Tweet.retweet_num==10))
print(sum(Company_Tweet_Tweet.retweet_num==11))
print(sum(Company_Tweet_Tweet.retweet_num==12))
print(sum(Company_Tweet_Tweet.retweet_num==13))
print(sum(Company_Tweet_Tweet.retweet_num==14))
print(sum(Company_Tweet_Tweet.retweet_num==15))
print(sum(Company_Tweet_Tweet.retweet_num==16))
print(sum(Company_Tweet_Tweet.retweet_num==17))
print(sum(Company_Tweet_Tweet.retweet_num==18))
print(sum(Company_Tweet_Tweet.retweet_num==19))
print(sum(Company_Tweet_Tweet.retweet_num==20))
print(sum(Company_Tweet_Tweet.retweet_num==21))

1203466
139841
36341
14776
7812
4671
3287
2232
1664
1254
990
843
667
562
472
392
381
342
301
258
280
206


In [ ]:
print(max(Company_Tweet_Tweet.retweet_num))
print(sum((Company_Tweet_Tweet.retweet_num>=0)&(Company_Tweet_Tweet.retweet_num<=10)))
print(sum((Company_Tweet_Tweet.retweet_num>=11)&(Company_Tweet_Tweet.retweet_num<=50)))
print(sum((Company_Tweet_Tweet.retweet_num>=51)&(Company_Tweet_Tweet.retweet_num<=100)))
print(sum((Company_Tweet_Tweet.retweet_num>=101)&(Company_Tweet_Tweet.retweet_num<=999)))

999
1416334
7321
889
457


In [ ]:
print(max(Company_Tweet_Tweet.like_num))
print(sum((Company_Tweet_Tweet.like_num>=0)&(Company_Tweet_Tweet.like_num<=10)))
print(sum((Company_Tweet_Tweet.like_num>=11)&(Company_Tweet_Tweet.like_num<=50)))
print(sum((Company_Tweet_Tweet.like_num>=51)&(Company_Tweet_Tweet.like_num<=100)))
print(sum((Company_Tweet_Tweet.like_num>=101)&(Company_Tweet_Tweet.like_num<=924)))

924
1404997
17320
1740
944
